# Import libraries

In [1]:
# drive access
from google.colab import drive
drive.mount('/content/drive')

# standard library
import numpy as np
import pandas as pd

# for audio
import librosa
from IPython.display import Audio

# for file streaming
import os

# for model
!pip install transformers
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.5 MB/s eta 0:00:00


In [2]:
# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load examples from slt

In [3]:
audio, _ = librosa.load('/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_slt_arctic/wav/arctic_b0539.wav', sr=16000)

In [4]:
Audio(audio, rate=16000)

In [5]:
audio, _ = librosa.load('/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_slt_arctic/wav/arctic_b0538.wav', sr=16000)

In [6]:
Audio(audio, rate=16000)

# get the files from bea_Amused folder & create auto transcription using wav2vec

In [7]:
audio_filename = os.listdir('/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_slt_arctic/wav')

audio_filename

['arctic_b0218.wav',
 'arctic_a0532.wav',
 'arctic_b0203.wav',
 'arctic_a0562.wav',
 'arctic_a0308.wav',
 'arctic_b0273.wav',
 'arctic_a0128.wav',
 'arctic_a0292.wav',
 'arctic_a0285.wav',
 'arctic_a0485.wav',
 'arctic_b0315.wav',
 'arctic_a0408.wav',
 'arctic_a0502.wav',
 'arctic_a0223.wav',
 'arctic_a0587.wav',
 'arctic_b0117.wav',
 'arctic_a0506.wav',
 'arctic_a0080.wav',
 'arctic_a0215.wav',
 'arctic_b0219.wav',
 'arctic_b0280.wav',
 'arctic_a0480.wav',
 'arctic_a0266.wav',
 'arctic_b0255.wav',
 'arctic_a0228.wav',
 'arctic_a0117.wav',
 'arctic_a0220.wav',
 'arctic_a0583.wav',
 'arctic_b0036.wav',
 'arctic_b0076.wav',
 'arctic_a0257.wav',
 'arctic_a0056.wav',
 'arctic_b0215.wav',
 'arctic_a0073.wav',
 'arctic_a0515.wav',
 'arctic_b0189.wav',
 'arctic_a0151.wav',
 'arctic_a0437.wav',
 'arctic_a0519.wav',
 'arctic_a0531.wav',
 'arctic_a0187.wav',
 'arctic_b0077.wav',
 'arctic_b0251.wav',
 'arctic_a0002.wav',
 'arctic_b0192.wav',
 'arctic_b0316.wav',
 'arctic_b0124.wav',
 'arctic_a051

In [8]:
audio_df = pd.DataFrame()

audio_df['filename'] = audio_filename

audio_df.head()

,filename
0,arctic_b0218.wav
1,arctic_a0532.wav
2,arctic_b0203.wav
3,arctic_a0562.wav
4,arctic_a0308.wav


In [9]:
auto_transcription = []

for filename in audio_df['filename']:
  audio, _ = librosa.load(f'/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_slt_arctic/wav/{filename}', sr=16000)
  input_value = processor(audio, return_tensors="pt", padding="longest", sampling_rate=16000).input_values
  predicted_id = torch.argmax(model(input_value).logits, dim=-1)
  transcription = processor.batch_decode(predicted_id)
  auto_transcription.append(transcription[0])
  del audio, input_value, predicted_id, transcription

auto_transcription

['THE ISSUE WAS NOT IN DOUBT',
 'MAY DROUGHT DESTROY YOUR CROPS',
 'A MONTH IN AUSTRALIA WOULD FINISH ME',
 'WHAT THE FLAMING',
 'HIS INFERNAL CHATTERING WORRIES ME EVEN NOW AS I THINK OF IT',
 'THEN IT WAS THAT A STRANGE THING HAPPENED',
 'THIS ONE HOPE WAS DESTROYED AS QUICKLY AS IT WAS BORN',
 'HERE IN THE MID MORNING THE FIRST CASUALTY OCCURRED',
 'BUT WHAT THEY WANT WITH YOUR TOOTH BRUSH IS MORE THAN I CAN IMAGINE',
 'EACH INSULT ADDED TO THE VALUE OF THE CLAIM',
 'AND THIS WAS THEIR SOLE CONVERSATION THROUGHOUT THE MEAL',
 'THE ELDER ROTTO EMPTIED ITS OCCUPANTS INTO THE STREET TO SEE THE TEST',
 'ANYTHING UNUSUAL OR ABNORMAL WAS SUFFICIENT TO SEND A FELLOW TO MOLOCHI',
 "MY I'M ALMOST HOMESICK FOR IT ALREADY",
 'HE HAD COMPARATIVELY NO ADVANTAGES AT FIRST',
 "THAT'S THORPE'S SAID THE YOUNG ENGINEER",
 'THE KLAWDIN WAS LEAVING NEXT MORNING FOR HANNALULU',
 'WHAT IF JEAN FAILED HIM',
 'THEY DIE OUT OF SPIGHT',
 "WELL THERE ARE BETTER MEN IN HAWAI THAT'S ALL",
 'AGAIN THE GIRLS APPL

confirm all files from bea_amused have been auto transcribed & add to the df

In [11]:
assert len(audio_df) == len(auto_transcription)

In [12]:
audio_df['auto_transcription'] = auto_transcription

In [13]:
audio_df.head()

,filename,auto_transcription
0,arctic_b0218.wav,THE ISSUE WAS NOT IN DOUBT
1,arctic_a0532.wav,MAY DROUGHT DESTROY YOUR CROPS
2,arctic_b0203.wav,A MONTH IN AUSTRALIA WOULD FINISH ME
3,arctic_a0562.wav,WHAT THE FLAMING
4,arctic_a0308.wav,HIS INFERNAL CHATTERING WORRIES ME EVEN NOW AS...


take a look at one example

In [14]:
example = audio_df['filename'].iloc[0]
audio, _ = librosa.load(f'/content/drive/MyDrive/266/Data/Raw_Data/Arctic/cmu_us_slt_arctic/wav/{example}', sr=16000)

In [15]:
Audio(audio, rate=16000)

In [16]:
audio_df['auto_transcription'].iloc[0]

'THE ISSUE WAS NOT IN DOUBT'

save the df to csv file for future use

In [17]:
audio_df.to_csv('/content/drive/MyDrive/266/Data/Clean_Data/Arctic/auto_transcription/slt.csv', index=False)

In [18]:
check = pd.read_csv('/content/drive/MyDrive/266/Data/Clean_Data/Arctic/auto_transcription/slt.csv')

check.head()

,filename,auto_transcription
0,arctic_b0218.wav,THE ISSUE WAS NOT IN DOUBT
1,arctic_a0532.wav,MAY DROUGHT DESTROY YOUR CROPS
2,arctic_b0203.wav,A MONTH IN AUSTRALIA WOULD FINISH ME
3,arctic_a0562.wav,WHAT THE FLAMING
4,arctic_a0308.wav,HIS INFERNAL CHATTERING WORRIES ME EVEN NOW AS...
